In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [2]:
query = """
SELECT *
FROM patients
"""
df = pd.read_sql(query, engine)
df

,subject_id,external_id,full_name,sex,date_of_birth,date_of_death
0,1,MRN-9001,Paciente Sin Fecha,F,None,None
1,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10,None
2,2,MRN-9002,None,M,1990-01-01,None


In [5]:
query = """
SELECT subject_id, external_id, full_name, sex, date_of_birth
FROM patients
ORDER BY external_id;
"""
df = pd.read_sql(query, engine)
df

,subject_id,external_id,full_name,sex,date_of_birth
0,1,MRN-9001,Paciente Sin Fecha,F,None
1,2,MRN-9002,None,M,1990-01-01
2,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10


In [7]:
pd.read_sql("""
SELECT subject_id, external_id, full_name, sex, date_of_birth
FROM patients
ORDER BY external_id;
""", engine)

,subject_id,external_id,full_name,sex,date_of_birth
0,1,MRN-9001,Paciente Sin Fecha,F,None
1,2,MRN-9002,None,M,1990-01-01
2,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10


# Pacientes sin fecha de nacimiento

In [15]:
query = """
SELECT COUNT(*) AS n_missing_dob
FROM patients
WHERE date_of_birth IS NULL;
"""
pd.read_sql(query, engine)

,n_missing_dob
0,1


In [28]:
query = """
SELECT COUNT(*) AS n_missing_name
FROM patients
WHERE full_name IS NULL;
"""
pd.read_sql(query, engine)

,n_missing_name
0,1


In [27]:
query = """
SELECT *
FROM admissions
"""
pd.read_sql(query, engine)

,hadm_id,subject_id,admittime,dischtime,admission_type,hospital_expire_flag
0,1,3,2101-10-01 08:00:00,2101-10-02 12:00:00,Emergency,False


In [26]:
query = """
SELECT *
FROM diagnoses
"""
pd.read_sql(query, engine)

,diagnosis_id,hadm_id,diagnosis_text
0,1,1,Guateeeemala referral note


In [30]:
import pandas as pd

query = """
SELECT COUNT(*) AS n_typo
FROM diagnoses
WHERE diagnosis_text ILIKE '%guate%';
"""

conn = engine.raw_connection()
try:
    df = pd.read_sql_query(query, conn)
finally:
    conn.close()

df


/tmp/ipykernel_2691/913755447.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,n_typo
0,1


In [31]:
import pandas as pd, sqlalchemy
pd.__version__, sqlalchemy.__version__


('2.1.1', '2.0.22')